In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.by import By

2023-04-10 11:10:30.401829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#helper function
def detect_and_predict_mask(frame, faceNet, maskNet):
    
    # grab the dimensions of the frame and then construct a blob from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return (locs, preds)

In [3]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector/deploy.prototxt"
weightsPath = r"face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

2023-04-10 11:10:49.618078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO] starting video stream...


In [4]:
# loop over the frames from the video stream
mask_framecount = 0
nomask_framecount = 0
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
        if mask>withoutMask:
            mask_framecount += 1
        else:
            nomask_framecount += 1

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Covered" if mask > withoutMask else "Uncovered"
        color = (0, 0, 255) if label == "Covered" else (0, 255, 0)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    print(mask_framecount, nomask_framecount)
    
    if mask_framecount >= 80:
        break
    key = cv2.waitKey(1) & 0xFF

#     # if the `q` key was pressed, break from the loop
#     if key == ord("q"):
#         break

(1, 1, 200, 7)
1/1 [==============================] - 1s 1s/step
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
0 1
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
0 2
(1, 1, 200, 7)
1/1 [==============================] - 0s 68ms/step
0 3
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
0 4
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
0 5
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
0 6
(1, 1, 200, 7)
1/1 [========================

1/1 [==============================] - 0s 152ms/step
30 129
(1, 1, 200, 7)
1/1 [==============================] - 0s 102ms/step
31 130
(1, 1, 200, 7)
1/1 [==============================] - 0s 98ms/step
32 131
(1, 1, 200, 7)
1/1 [==============================] - 0s 102ms/step
32 133
(1, 1, 200, 7)
1/1 [==============================] - 0s 103ms/step
33 134
(1, 1, 200, 7)
1/1 [==============================] - 0s 103ms/step
34 135
(1, 1, 200, 7)
1/1 [==============================] - 0s 106ms/step
35 136
(1, 1, 200, 7)
1/1 [==============================] - 0s 97ms/step
36 137
(1, 1, 200, 7)
1/1 [==============================] - 0s 96ms/step
37 138
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
37 139
(1, 1, 200, 7)
1/1 [==============================] - 0s 101ms/step
37 141
(1, 1, 200, 7)
1/1 [==============================] - 0s 111ms/step
38 142
(1, 1, 200, 7)
1/1 [==============================] - 0s 99ms/step
39 143
(1, 1, 200, 7)
1/1 [=========================

In [5]:
if mask_framecount >= 80:
    driver = webdriver.Chrome(r'E:\hotstar python\chromedriver')
    driver.maximize_window()

    driver.get("https://blynk.cloud/dashboard/69761/global/filter/879801/organization/69761/devices/596041/dashboard")
    time.sleep(5)

    email=driver.find_element(By.XPATH,"/html/body/div/div/div[1]/div[4]/div[2]/form/div[2]/div[2]/div/div/div/div/div/span/input")
    password=driver.find_element(By.XPATH,"/html/body/div/div/div[1]/div[4]/div[2]/form/div[3]/div[2]/div/div/div/div/div/span/input")
    login_button=driver.find_element(By.XPATH,"/html/body/div/div/div[1]/div[4]/div[2]/form/div[5]/div/div/div/div/button")
    email.send_keys("ysoni9109@gmail.com")
    password.send_keys("Abcdef@1")
    login_button.click()
    time.sleep(10)

    m1=driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[4]/div/div[2]/section/section/main/section/section/main/div/div/div/div/div[8]/div[2]/div/div/div/div/div/div[2]/div/div[1]/div/div[2]/div/button")
    m2=driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[4]/div/div[2]/section/section/main/section/section/main/div/div/div/div/div[8]/div[2]/div/div/div/div/div/div[2]/div/div[2]/div/div[2]/div/button")
    integer_v2=driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[4]/div/div[2]/section/section/main/section/section/main/div/div/div/div/div[8]/div[2]/div/div/div/div/div/div[2]/div/div[3]/div/div[2]/div/button")

    m1.click()
    m2.click()
    integer_v2.click()

/var/folders/k0/dv_by81x393f2f6tgfhykf900000gn/T/ipykernel_971/1394135721.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'E:\hotstar python\chromedriver')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div[4]/div/div[2]/section/section/main/section/section/main/div/div/div/div/div[8]/div[2]/div/div/div/div/div/div[2]/div/div[1]/div/div[2]/div/button"}
  (Session info: chrome=111.0.5563.146)
Stacktrace:
0   chromedriver                        0x00000001039a5428 chromedriver + 4899880
1   chromedriver                        0x0000000103922a23 chromedriver + 4364835
2   chromedriver                        0x000000010356cbf6 chromedriver + 474102
3   chromedriver                        0x00000001035b04f0 chromedriver + 750832
4   chromedriver                        0x00000001035b0751 chromedriver + 751441
5   chromedriver                        0x00000001035f4834 chromedriver + 1030196
6   chromedriver                        0x00000001035d658d chromedriver + 906637
7   chromedriver                        0x00000001035f1b5b chromedriver + 1018715
8   chrom

In [ ]:
if mask_framecount >=  80:
    #x,y=map(int,input("Enter Time in Hr(space)Min  [24 hrs format]: ").split())
    t = (time.ctime()).split()[3]
    hour = t.split(':')[0]
    minutes = t.split(':')[1]
    seconds = t.split(':')[2]
    pywhatkit.sendwhatmsg("{}".format('+919300029388'),'Thief Detected: Taser system is getting activated',int(hour),int(minutes)+1)

In [ ]:
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()